## 一，Autograph和tf.Module概述


前面在介绍Autograph的编码规范时提到构建Autograph时应该避免在@tf.function修饰的函数内部定义tf.Variable. 

但是如果在函数外部定义tf.Variable的话，又会显得这个函数有外部变量依赖，封装不够完美。

一种简单的思路是定义一个类，并将相关的tf.Variable创建放在类的初始化方法中。而将函数的逻辑放在其他方法中。

这样一顿猛如虎的操作之后，我们会觉得一切都如同人法地地法天天法道道法自然般的自然。

惊喜的是，TensorFlow提供了一个基类tf.Module，通过继承它构建子类，我们不仅可以获得以上的自然而然，而且可以非常方便地管理变量，还可以非常方便地管理它引用的其它Module，最重要的是，我们能够利用tf.saved_model保存模型并实现跨平台部署使用。

实际上，tf.keras.models.Model,tf.keras.layers.Layer 都是继承自tf.Module的，提供了方便的变量管理和所引用的子模块管理的功能。

**因此，利用tf.Module提供的封装，再结合TensoFlow丰富的低阶API，实际上我们能够基于TensorFlow开发任意机器学习模型(而非仅仅是神经网络模型)，并实现跨平台部署使用。**

## 二，应用tf.Module封装Autograph

In [1]:
import tensorflow as tf
x = tf.Variable(1.0, dtype=tf.float32)

@tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.float32)])
def add_print(a):
    x.assign_add(a)
    tf.print(x)
    return (x)

add_print(tf.constant(3.0))

4


<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

In [2]:
class DemoModule(tf.Module):
    def __init__(self, init_value=tf.constant(0.0), name=None):
        super(DemoModule, self).__init__(name=name)
        with self.name_scope:
            self.x = tf.Variable(init_value, dtype=tf.float32, trainable=True)

    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.float32)])
    def addprint(self, a):
        with self.name_scope:
            self.x.assign_add(a)
            tf.print(self.x)
            return self.x

demo = DemoModule(init_value=tf.constant(1.0, dtype=tf.float32))
demo.addprint(tf.constant(5.0, dtype=tf.float32))

6


<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [3]:
demo.variables, demo.trainable_variables

((<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,),
 (<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,))

In [4]:
#查看模块中的全部子模块
demo.submodules

()

In [5]:
tf.saved_model.save(demo, './demo/1', signatures={"serving_default":demo.addprint})

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./demo/1\assets


In [6]:
#加载模型
demo2 = tf.saved_model.load("./demo/1")
demo2.addprint(tf.constant(5.0))

11


<tf.Tensor: shape=(), dtype=float32, numpy=11.0>

In [7]:
# 查看模型文件相关信息，红框标出来的输出信息在模型部署和跨平台使用时有可能会用到
!saved_model_cli show --dir ./demo/1 --all

2020-09-16 16:16:53.583791: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_101.dll
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['a'] tensor_info:
        dtype: DT_FLOAT
        shape: ()
        name: serving_default_a:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: ()
        name: Statef

In [10]:
mymodule = tf.Module()
mymodule.x = tf.Variable(0.0)

@tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.float32)])
def addprint(a):
    mymodule.x.assign_add(a)
    tf.print(mymodule.x)
    return mymodule.x

mymodule.addprint = addprint

In [11]:
mymodule.addprint(tf.constant(1.0)).numpy()

1


1.0

In [12]:
print(mymodule.variables)

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>,)


In [16]:
#使用tf.saved_model 保存模型
tf.saved_model.save(mymodule,"./mymodule",
    signatures = {"serving_default":mymodule.addprint})

#加载模型
mymodule2 = tf.saved_model.load("./mymodule")
mymodule2.addprint(tf.constant(5.0))

INFO:tensorflow:Assets written to: ./mymodule\assets
6


<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

## 三，tf.Module和tf.keras.Model，tf.keras.layers.Layer

In [17]:
import tensorflow as tf
from tensorflow.keras import models,layers,losses,metrics

In [19]:
print(issubclass(tf.keras.Model,tf.Module))
print(issubclass(tf.keras.layers.Layer,tf.Module))
print(issubclass(tf.keras.Model,tf.keras.layers.Layer))

True
True
True


In [20]:
tf.keras.backend.clear_session()

model = models.Sequential()
model.add(layers.Dense(4,input_shape = (10,)))
model.add(layers.Dense(2))
model.add(layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.variables

[<tf.Variable 'dense/kernel:0' shape=(10, 4) dtype=float32, numpy=
 array([[-0.08844614,  0.17002028, -0.4901232 ,  0.06460047],
        [ 0.07446593,  0.4274279 ,  0.41890955,  0.30275506],
        [ 0.19276202, -0.06372249, -0.43789458, -0.31411454],
        [-0.27371576,  0.3351133 ,  0.5749022 ,  0.1318059 ],
        [ 0.31592053, -0.18724221,  0.06485921,  0.38987422],
        [ 0.06443405,  0.00983202, -0.4250726 ,  0.36896765],
        [ 0.60787237, -0.4419867 , -0.64729315, -0.09726304],
        [ 0.2793491 ,  0.41111314, -0.36566406,  0.50874865],
        [-0.5377642 , -0.04290181,  0.04355782,  0.41110516],
        [ 0.32869267, -0.12562257, -0.5047242 , -0.11344445]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[-0.45435834,  0.7754381 ],
        [-0.38701987,  0.16969466],
        [ 0.23731756,  0.5922353 ],
        [-

In [22]:
model.layers[0].trainable = False #冻结第0层的变量,使其不可训练
model.trainable_variables

[<tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[-0.45435834,  0.7754381 ],
        [-0.38701987,  0.16969466],
        [ 0.23731756,  0.5922353 ],
        [-0.40114498, -0.39137983]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'dense_2/kernel:0' shape=(2, 1) dtype=float32, numpy=
 array([[ 0.4378599],
        [-0.4187923]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [23]:
model.submodules

(<tensorflow.python.keras.engine.input_layer.InputLayer at 0x1c0f225f5c8>,
 <tensorflow.python.keras.layers.core.Dense at 0x1c0f252ca88>)

In [24]:
model.layers

In [25]:
print(model.name)
print(model.name_scope())

sequential
sequential
